In [1]:
from Utility import blockproc
import numpy as np
from PIL import Image
from scipy import ndimage

In [2]:
colaImage = Image.open("cola.jpg")
colaImageMat = np.array(colaImage)
colaImageMat.shape

(640, 640, 3)

In [3]:
blockproc(colaImageMat[:,:,0], [10,10], np.mean)

array([[180.01, 180.01, 180.01, ..., 173.71, 173.71, 173.71],
       [180.01, 180.01, 180.01, ..., 173.71, 173.71, 173.71],
       [180.01, 180.01, 180.01, ..., 173.71, 173.71, 173.71],
       ...,
       [209.42, 209.42, 209.42, ..., 212.92, 212.92, 212.92],
       [209.42, 209.42, 209.42, ..., 212.92, 212.92, 212.92],
       [209.42, 209.42, 209.42, ..., 212.92, 212.92, 212.92]])


# 1.1 Optimal Color Space

In [4]:
def colortrans(imgMat):
    """
    imgMat: rgb image as a 3D np array
    ------
    return
    ycbcr: ycbcr image as a 3D np array
    """
    shape = imgMat.shape
    ycbcr = np.zeros(shape)
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            
            l = np.array([0,128,128])
            rgb = np.array([imgMat[i,j,0], imgMat[i,j,1], imgMat[i,j,2]])
            transMat = np.array([[.299,.587,.114],[-.168736,-.331264,.5],[.5,-.418688,-0.081312]])
            
            ycbcr[i,j,:] += l + np.matmul(transMat, rgb)
            
    return ycbcr

In [5]:
def invcolortrans(imgMat):
    """
    imgMat: ycbcr image as 3D np array
    --------
    return
    rgb: rgb image as 3D np array
    """
    shape = imgMat.shape
    rgb = np.zeros(shape)
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            
            l = np.array([0,128,128])
            ycbcr = np.array([imgMat[i,j,0], imgMat[i,j,1], imgMat[i,j,2]])
            transMat = np.array([[.299,.587,.114],[-.168736,-.331264,.5],[.5,-.418688,-0.081312]])
            transMat = np.linalg.inv(transMat)
            
            diff = ycbcr - l
            rgb[i,j,:] += np.matmul(transMat, diff)
            
    return rgb

In [6]:
ycbcr = colortrans(colaImageMat)
rgb = invcolortrans(ycbcr)

In [7]:
np.sum(rgb - colaImageMat)

-3.6895552012757565e-08

# 1.2 Downsample

In [8]:
def downsample(imgChannel, w):
    
    meanImg = blockproc(imgChannel, [w,w], np.mean)
    shape = imgChannel.shape
    downsampleImg = np.zeros((shape[0]//w, shape[1]//w))
    
    for i in range(0, shape[0]//w):
        for j in range(0,shape[1]//w):
            
            downsampleImg[i,j] = meanImg[i+ w//2, j+w//2]
            
    return downsampleImg

In [11]:
print(downsample(ycbcr[:,:,1],6).shape)

(106, 106)


# Continue

In [ ]:
def downsample(img, w):
    """
    img: ycbcr img as 3D np array
    ------
    return
    ycbcr: same as input with cb and cr channels downsampled
    """
    
    downsampled_cb = blockproc(img[:,:,1], [w,w], np.mean)
    downsampled_cr = blockproc(img[:,:,2], [w,w], np.mean)
    
    ycbcr = np.array([img[:,:,0], downsampled_cb, downsampled_cr])
    
    return ycbcr

In [ ]:
def downsample(image, w):
    """
    image: ycbcr img as 3D np array
    ------
    return
    ycbcr: same as input with cb and cr channels downsampled
    """
    
    x,y = image[:,:,1].shape
    bx = w
    by = w
    
    out = np.zeros((bx,by,2))
    
    for channel in range(1,3):
        for i in range(0,bx,1):
            for j in range(0,by,1):

                out[i, j, channel-1] = np.mean(image[i:i+w,j:j+w, channel])
    
    
#     out[:,:,0] = image[:,:,0]
    
    return out

In [ ]:
downsample(ycbcr, 100).shape

In [ ]:
np.sum(ycbcr[:,:,1]) == np.sum(downsample(ycbcr, 10)[:,:,1])

In [ ]:
scipy.ndi

In [ ]:
def block_mean(ar, fact):
    assert isinstance(fact, int), type(fact)
    sx, sy = ar.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    regions = sy/fact * (X/fact) + Y/fact
    res = ndimage.mean(ar, labels=regions, index=np.arange(regions.max()))
    res.shape = (int(sx/fact), int(sy/fact))
    return res

In [ ]:
block_mean(ycbcr[:,:,1],100).shape

In [ ]:
import numpy as np
from scipy import ndimage

def block_mean(ar, fact):
    assert isinstance(fact, int), type(fact)
    sx, sy = ar.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    regions = sy/fact * (X/fact) + Y/fact
    res = ndimage.mean(ar, labels=regions, index=np.arange(regions.max() + 1))
    res.shape = (sx/fact, sy/fact)
    return res

ar = np.random.rand(20000).reshape((100, 200))
block_mean(ar, 5).shape  # (20, 40)

In [ ]:
def upsample(img, w):
    """
    img: ycbcr img as 3D np array
    ------
    return
    ycbcr: same as input with cb and cr channels upsampled
    """